In [1]:
import tkinter as tk
from tkinter import ttk,Canvas
#from ttkbootstrap.constants import *
import time
import funciones as fn
import threading
from os.path import isfile
import csv
import datetime

In [2]:
#Funciones
def conectarsrv():
    '''Gestiona la conexión con el servidor y los estados'''
    global estados, cn, tempcolor
    txt=app.btn_conectar_srv['text']
    app.lbl_gral2.config(text='...', **neutro)
    if txt=='CONECTAR\nSERVIDOR':
        try:
            #srv.conectar(host="172.16.4.2", port=9999)
            srv.conectar(host="192.168.99.10", port=9999)
            hs=""
            hs=srv.enviar("hs")
            if hs=="hs":
                tempcolor=green
                srv.status="Conectado"
                app.btn_conectar_srv.config(text='SERVER\nCONECTADO', **verde)
                app.lbl_gral1.config(text='Conexión con servidor OK', **neutro)
                
            else:
                tempcolor='red'
                srv.status="Error"
                app.lbl_gral1.config(text='Error en datos de handshake', **neutro)
                app.btn_conectar_srv.config(text='CONECTAR\nSERVIDOR', **rojo)
        except Exception as e:
            tempcolor='red'
            srv.status="Error de conexión"
            app.lbl_gral1.config(text=str(e), **neutro)
            app.btn_conectar_srv.config(text='CONECTAR\nSERVIDOR', **rojo)
    else:
        tempcolor=standar
        srv.cerrar()
        srv.status='Desconectado'
        app.btn_conectar_srv.config(text='CONECTAR\nSERVIDOR', **neutro)
        cn=0
    estados['srvcon']=srv.status
    
def conectarlcr():
    app.lbl_gral2.config(text='...', **neutro)
    txt=app.btn_conectar_LCR['text']
    global LCR,tempcolor
    if txt == "CONECTAR\nLCR":
        sts=LCR.conectar(port='COM3', baudrate=19200, bytesize=8, parity='N', stopbits=1, timeout=10)
        if sts==1:
            tempcolor=green
            LCR.inicializado=True
            estados['sercon']='Conectado'
            app.btn_conectar_LCR.config(text='LCR\nCONECTADO', **verde)
            app.lbl_gral1.config(text='Conexión con LCR OK', **neutro)
        elif sts==0:
            tempcolor='red'
            estados['sercon']='Error'
            app.btn_conectar_LCR.config(text='CONECTAR\nLCR', **rojo)
            app.lbl_gral1.config(text='No se encuentra puerto', **neutro)
        else:
            tempcolor='red'
            estados['sercon']='Error'
            app.btn_conectar_LCR.config(text='CONECTAR\nLCR', **rojo)
            app.lbl_gral1.config(text=LCR.estado['conexion'], **neutro)
    else:
        LCR.desconectar()
        tempcolor=standar
        LCR.inicializado=False
        estados['sercon']='Desconectado'
        app.btn_conectar_LCR.config(text='CONECTAR\nLCR', **neutro)
        app.lbl_gral1.config(text='LCR Desconectado', **neutro)


def configurar():
    app.lbl_gral2.config(text='...', **neutro)
    global estados, tfrec, ifrec, repeticion, frecuencias, repeticiones, tempcolor
    dic={True:"OK", False: "NO OK"}
    ac=fn.configurar()
    frecuencias,repeticiones,cnferror=ac.cargarconfig("configuracion/config.txt")
    if cnferror=='no':
        tempcolor=standar
        estados['config']='Configurado'
        app.lbl_gral1.config(text=f'frec:{frecuencias}, rep:{repeticiones}', **neutro)
        app.btn_configurar.config(**neutro)
    else:
        tempcolor='red'
        estados['config']='Error'
        app.lbl_gral1.config(text=str(cnferror), **rojo)
        app.btn_configurar.config(**rojo)
    
    tfrec=len(frecuencias)
    ifrec=0
    repeticion=0
    
    

def inimed():
    "Inicia y detiene la medicion"
    global flag_medIni
    app.lbl_gral2.config(text='...', **neutro)
    txt=app.btn_medir['text']
    if txt == "INICIAR\nMEDICIÓN":
        estados['medicion']='Iniciada'
        flag_medIni=True
        app.btn_medir.config(text='MEDICIÓN\nEN MARCHA', **verde)
    else:
        estados['medicion']='Detenida'
        flag_medIni=False
        app.btn_medir.config(text='INICIAR\nMEDICIÓN', **neutro)
  

#funcion para guardar una fila en csv.
#si el archivo no existe, te lo crea
def escribir_csv(archivo:str, datos:list, encabezado:list):
    """
    Abre un archivo CSV en modo 'append', por lo que agrega una nueva fila.
    archivo     -> string. Formato 'carpeta\\archivo.csv' ('Mediciones\probando.csv')
    fila        -> lista con los valores a guardar
    encabezado  -> Lista con los textos de los encabezados
    """

    aux=isfile(archivo)
    
    with open(archivo, mode='a', newline='',) as archivo_csv:
        writer = csv.writer(archivo_csv, delimiter=";")
        if not aux:
            writer.writerow(encabezado)
        writer.writerow(datos)
        
    archivo_csv.close()



In [3]:
#Funcion para escribir el estado
def informacion():
    resultado=''
    if flag_medIni:
        if flag_posOK:
            if flag_cicloFin: resultado='Esperando nueva posición'
            else: resultado='Medición en proceso'
        else: resultado='Buscando nueva posición'
    else:
        if estados['config']=='Configurado':
            if estados['srvcon']=='Conectado':
                if estados['sercon']=='Conectado': resultado='Listo para comenzar a medir'
                else: resultado='El equipo serie se encuentra desconectado'
            else: resultado='El servidor se encuentra desconectado'
        else: resultado='Falta cargar configuración'
    app.lbl_gral2.config(text=resultado, **neutro)

        


In [4]:
#Funciones dummy
import random
def dummyconectarlcr():

    app.lbl_gral2.config(text='...', **neutro)
    txt=app.btn_conectar_LCR['text']
    global LCR,tempcolor
    if txt == "CONECTAR\nLCR":
        sts=random.randint(0,1)
        if sts==1:
            tempcolor=green
            LCR.inicializado=True
            estados['sercon']='Conectado'
            app.btn_conectar_LCR.config(text='LCR\nCONECTADO', **verde)
            app.lbl_gral1.config(text='Conexión con LCR OK', **neutro)
        elif sts==0:
            tempcolor='red'
            estados['sercon']='Error'
            app.btn_conectar_LCR.config(text='CONECTAR\nLCR', **rojo)
            app.lbl_gral1.config(text='No se encuentra puerto', **neutro)
        else:
            tempcolor='red'
            estados['sercon']='Error'
            app.btn_conectar_LCR.config(text='CONECTAR\nLCR', **rojo)
            app.lbl_gral1.config(text=LCR.estado['conexion'], **neutro)
    else:
        #LCR.desconectar()
        tempcolor=standar
        LCR.inicializado=False
        estados['sercon']='Desconectado'
        app.btn_conectar_LCR.config(text='CONECTAR\nLCR', **neutro)
        app.lbl_gral1.config(text='LCR Desconectado', **neutro)


def dummyconectarsrv():
    '''Gestiona la conexión con el servidor y los estados'''
    global estados, cn, tempcolor
    txt=app.btn_conectar_srv['text']
    app.lbl_gral2.config(text='...', **neutro)
    if txt=='CONECTAR\nSERVIDOR':
        try:
            #srv.conectar(host="172.16.4.2", port=9999)
            srv.conectar(host="192.168.99.10", port=9999)
            hs=""
            hs=srv.enviar("hs")
            if hs=="hs":
                tempcolor=green
                srv.status="Conectado"
                app.btn_conectar_srv.config(text='SERVER\nCONECTADO', **verde)
                app.lbl_gral1.config(text='Conexión con servidor OK', **neutro)
                
            else:
                tempcolor='red'
                srv.status="Error"
                app.lbl_gral1.config(text='Error en datos de handshake', **neutro)
                app.btn_conectar_srv.config(text='CONECTAR\nSERVIDOR', **rojo)
        except Exception as e:
            tempcolor='red'
            srv.status="Error de conexión"
            app.lbl_gral1.config(text=str(e), **neutro)
            app.btn_conectar_srv.config(text='CONECTAR\nSERVIDOR', **rojo)
    else:
        srv.cerrar()
        srv.status='Desconectado'
        app.btn_conectar_srv.config(text='CONECTAR\nSERVIDOR', **neutro)
        cn=0
    estados['srvcon']=srv.status

In [5]:
class App(threading.Thread):

    def __init__(self):
        threading.Thread.__init__(self)
        #self.daemon=True
        self.start()

    def callback(self):
        self.root.quit()

    def btn_enter(self,event):
        global tempcolor
        tempcolor=event.widget['bg']
        if event.widget['state'] != tk.DISABLED:
            event.widget.config(bg=blue)
        else:
            pass

    def btn_leave(self,event):
        global tempcolor
        event.widget.config(bg=tempcolor)

    def popup(self):
        if self.root['width']==800:
            self.root.config(width=549)
            self.canvas.configure(width=549)
        else:
            self.root.config(width=800)
            self.canvas.configure(width=800)

    def run(self):
        self.root = tk.Tk()
        self.root.protocol("WM_DELETE_WINDOW", self.callback)
        self.root.title("ADQUISIDOR CAPEM")
        self.root.config(width=549, height=375,bg='#f5f5f5')
        self.canvas=Canvas(self.root,width=549, height=375)

        #BOTONES
        self.btn_configurar= tk.Button(text="CARGAR\nCONFIGURACION", command=configurar,relief="groove", activebackground='#9ACBFB', **neutro)
        self.btn_configurar.place(x=20, y=30,width=110, height=40)
        self.btn_configurar.bind('<Enter>',self.btn_enter)
        self.btn_configurar.bind('<Leave>',self.btn_leave)

        self.btn_conectar_srv= tk.Button(text="CONECTAR\nSERVIDOR", command=conectarsrv,relief="groove", **neutro)
        self.btn_conectar_srv.place(x=20, y=70,width=110, height=40)
        self.btn_conectar_srv.bind('<Enter>',self.btn_enter)
        self.btn_conectar_srv.bind('<Leave>',self.btn_leave)

        self.btn_conectar_LCR= tk.Button(text="CONECTAR\nLCR", command=dummyconectarlcr,relief="groove", **neutro)
        self.btn_conectar_LCR.place(x=20, y=110,width=110, height=40)
        self.btn_conectar_LCR.bind('<Enter>',self.btn_enter)
        self.btn_conectar_LCR.bind('<Leave>',self.btn_leave)

        self.btn_medir= tk.Button(text="INICIAR\nMEDICIÓN", command=inimed, relief="groove", **neutro, state=tk.DISABLED)
        self.btn_medir.place(x=20, y=210,width=110, height=90)
        self.btn_medir.bind('<Enter>',self.btn_enter)
        self.btn_medir.bind('<Leave>',self.btn_leave)
        
        self.btn_info= tk.Button(text='INFO', command=self.popup,relief="groove", **neutro)
        self.btn_info.place(x=480, y=30,width=50, height=50)

        #SEMAFORO

        self.btn_SEMAFORO= tk.Button(text="CONFIGURANDO", command=informacion, relief="flat", bg=gray, fg='black' ,font=("Arial",13, 'bold'))
        self.btn_SEMAFORO.place(x=200, y=30,width=270, height=270)


        #ETIQUETAS DE INFO
        self.lbl_gral1=ttk.Label(text="...", padding=0, relief="sunken",width=77,**neutro)
        self.lbl_gral1.place(x=1, y=330)      

        self.lbl_gral2=ttk.Label(text="...", padding=0, relief="sunken",width=77,**neutro)
        self.lbl_gral2.place(x=1, y=353)  

        #Info adicional
        self.lbl_1=ttk.Label(relief="sunken",width=25,**neutro)
        self.lbl_1.place(x=557, y=20)
        self.lbl_2=ttk.Label(relief="sunken",width=25,**neutro)
        self.lbl_2.place(x=557, y=50)
        self.lbl_3=ttk.Label(relief="sunken",width=25,**neutro)
        self.lbl_3.place(x=557, y=80)
        self.lbl_4=ttk.Label(relief="sunken",width=25,**neutro)
        self.lbl_4.place(x=557, y=110)
        self.lbl_5=ttk.Label(relief="sunken",width=25,**neutro)
        self.lbl_5.place(x=557, y=140)
        self.lbl_6=ttk.Label(relief="sunken",width=25,**neutro)
        self.lbl_6.place(x=557, y=170)
        self.lbl_7=ttk.Label(relief="sunken",width=25,**neutro)
        self.lbl_7.place(x=557, y=200)
        self.lbl_8=ttk.Label(relief="sunken",width=25,**neutro)
        self.lbl_8.place(x=557, y=230)
        self.canvas.create_line(552,20,552,350)
        self.canvas.pack(expand=tk.YES)

        self.root.mainloop()
    
    

In [6]:
flag_medIni=False       #Flag para iniciar medicion desde HMI (A)
flag_posOK=False        #Flag para indicar que la posicion está OK (B). Se actualiza mediante OPC
flag_comOK= False       #Flag que indic que la comunicación es actual y está OK (C)
flag_cicloFin=False     #Flag para indicar cuándo finalizó las n repeticiones con todas las frecuencias

frecuencias=[]          #En esta lista se cargan las frecuencias.
repeticiones=0          #Se carga el numero de repeticiones

srv=fn.conexionsrv()    #Genero el objeto servidor, que tiene todas las funciones para conectarse
cn=0                    #cn es el numero de control que se usa para chequear que la info leida  de srv es la correcta 
LCR=fn.LCR()            #creo el objeto lcr

#Variables que se van actualizando con los barridos de frecuencia
tfrec=0                 #cantidad de frecuencias totales en la config
ifrec=0                 #Indice de frecuencia actual
repeticion=0            #repeticion actual
filtroPos01=0           #Fitro para determinar el cambio de posicion OK a no OK
filtroPos10=0           #Fitro para determinar el cambio de posicion no OK a OK

estados=dict(srvcon="Desconectado",sercon="Desconectado",   #diccionario  para indicar los estados de las conexiones y funciones
             config="Sin configurar", medicion="Detenido")

encabezado=['Repetición', 'Frecuencia','En posicion','Posicion', 'Error SC', 'Presion', 'Nivel agua', 
            'T recipiente', 'T subenfriada', 'T Estimada', 'Inductancia','Unidad', 'Resistencia', 'unidad', 'Dia', 'Hora']

datos_guardar=[]

#Diccionarios creados para dar estilos a los labels
verde={'background':'#4BF24B', 'foreground':'black','font':("Arial",9,'bold')}
rojo={'background':'red', 'foreground':'white','font':("Arial",9,'bold')}
amarillo={'background':'#ffe000', 'foreground':'black','font':("Arial",9,'bold')}
neutro={'background':'#f0f0f0', 'foreground':'black','font':("Arial",9)}
standar='#f0f0f0'
green="#4BF24B"
yellow='#ffe000'
gray='#c0c0c0'
blue='#D7EAFF'
tempcolor=''
bgcolor=blue

In [7]:
app = App()

In [8]:
configurar()
while app.is_alive():       #Se finaliza cuando la interfaz se cierra
    #Habilita o deshabilita el boton de medir en función de los estados de las conexiones
    if estados['srvcon']=='Conectado' and estados['sercon']=='Conectado' and estados['config']=='Configurado':
        app.btn_medir.config(state=tk.NORMAL)
    else:
        app.btn_medir.config(text="INICIAR\nMEDICIÓN", state=tk.DISABLED)
        flag_medIni=False

    if flag_medIni:         #Iniciar medición desde HMI (A)
        app.btn_configurar['state']=tk.DISABLED
        app.btn_conectar_srv['state']=tk.DISABLED
        app.btn_conectar_LCR['state']=tk.DISABLED
        #Pido datos a OPC:
        cn+=1               #mando un numero de confirmación para que lo devuelva y saber si el valor devuelto es actual
        if cn>999: cn=1
        dts=srv.enviar(str(cn))
        if dts=='error':
            app.lbl_gral1.config(text=srv.status, **rojo)
            app.btn_conectar_srv.config(text= 'CONECTAR\nSERVIDOR',**rojo)
            estados['srvcon']='Error'
            continue                    #Con continue vuelvo al while, saltando todo lo de abajo
        
        datosOPC=list(dts[1:-1].split(sep=","))     #Hago un arreglo para quitarle los corchetes de la lista
        for n,x in enumerate(datosOPC):             #y los convierto en numeros 
            if n>1:                           
                datosOPC[n]=float(x)
            else:
                datosOPC[n]=int(x)             #el primero queda entero porque lo comparo con cn

        #Analizo el dato de posición traido de OPC. Si está en posición, se activa un flag (B)
        if datosOPC[1]==1:         #El bit de posición tiene que estar en el lugar 1 de la lista enviada
            filtroPos01+=1
            if filtroPos01>=10:     #Espera un tiempo... considerando las demoras, filtroPos01>=10 es un poco menos de 5 segundos
                flag_posOK=True
        else:
            flag_posOK=False
            filtroPos01=0
        
        #Comparo el numero de confirmación para chequear comunicacion (C)
        if datosOPC[0]==cn and datosOPC[0]>0:        #Activa el flag comOK si esta todo OK
            #app.lbl_stssrv.config(text="Conectado")
            app.lbl_1.config(text=f'En posicion: {datosOPC[1]}')
            app.lbl_2.config(text=f'Posicion: {datosOPC[2]}')
            app.lbl_3.config(text=f'Error SC: {datosOPC[3]}')
            app.lbl_4.config(text=f'Presion: {datosOPC[4]}')
            app.lbl_5.config(text=f'Nivel: {datosOPC[5]}')
            flag_comOK=True
        else:
            flag_comOK=False    #Si no lo desactiva y deja un mensaje en HMI
            app.lbl_gral1.config(text="Error de datos desde OPC",**rojo)
        

        #Chequeo cuándo se cumplen las n repeticiones de los barridos de frecuencia (D)
        if repeticion>=repeticiones:
            flag_cicloFin=True
        app.lbl_7.config(text=f'Repetición: {repeticion}')
        app.lbl_8.config(text=f'PosOK: {flag_posOK}, ComOK:{flag_comOK}, CicloFin:{flag_cicloFin}',width=40)

        if flag_cicloFin:           #Si se finaliza el ciclo...
            repeticion=0
            ifrec=0        
            if not flag_posOK:      #...Espera a que salga de la posición para reiniciarlo
                filtroPos10+=1
                if filtroPos10>=10:    #Espera un tiempo... considerando las demoras, filtroPos10>=10 es un poco menos de 5 segundos
                    flag_cicloFin=False
                    filtroPos10=0
            else: filtroPos10=0

        
    
        # Chequeo que se den las condiciones para medir (A=1 -siempre está porque es el if principal- , B=1 ,C=1 y D=0)
        if flag_posOK and flag_comOK and not flag_cicloFin:
                        
            #Si se cumplen las condiciones, empiezo a medir
            app.lbl_gral1.config(text='Midiendo, no mover', **rojo)
            
            #Primero traigo la frecuencia configurada
            frecuencia=frecuencias[ifrec]      #no hago un 'for x in frecuencias' porque no pararía hasta terminar todos los valores
            app.lbl_6.config(text=f'Frecuencia: {frecuencia}')
            LCR.cambiarfreq(frecuencia)
            time.sleep(1)
            datosLCR=LCR.medir()
            
            #armo la cadena de datos para guardar, con los datos de OPC y de equipo a medir
            fecha=[str(datetime.datetime.now().date()), str(datetime.datetime.now().time())]
            
            datos_guardar=[repeticion, frecuencia]       #datos básicos
            datos_guardar=datos_guardar+datosOPC[1:]     #Datos de OPC a partir del 1 (el cero es el check de comunicacion)
            datos_guardar=datos_guardar+datosLCR         #Datos de LCR
            datos_guardar=datos_guardar+fecha            #Fecha y hora

            #Guardo los datos en csv
            escribir_csv('Mediciones\datos.csv',datos_guardar,encabezado)
            ifrec+=1
            if ifrec>=tfrec:        #límite para las frecuencias
                ifrec=0
                repeticion+=1
            
                
        #Colores y textos para la etiqueta general
        if flag_cicloFin:
            app.btn_SEMAFORO.config(text='FIN DE MEDICIÓN\nSE PUEDE MOVER',background=green)
        
        if flag_posOK and not flag_cicloFin:
            #app.canvas.config(background=yellow)
            app.btn_SEMAFORO.config(text='MIDIENDO\nNO MOVER',background='red')
        if not flag_posOK:
            app.btn_SEMAFORO.config(text='BUSCANDO\nPOSICION',background=yellow)
        
    else:
        if estados['srvcon']=='Conectado' and estados['sercon']=='Conectado' and estados['config']=='Configurado':
            app.btn_SEMAFORO.config(text='LISTO PARA\nMEDIR',background='white')
        else:
            app.btn_SEMAFORO.config(text='CONFIGURANDO',background=gray)
        app.btn_configurar['state']=tk.NORMAL
        app.btn_conectar_srv['state']=tk.NORMAL
        app.btn_conectar_LCR['state']=tk.NORMAL

del app
#Cuando cierra la aplicacion, manda el comando de cerrar el servidor para que no de problemas
#srv.cerrar()
        

AttributeError: 'LCR' object has no attribute 'cambiarfreq'